In [1]:
import numpy as np 
import torch 
import torch.nn as nn 
import torch.nn.functional as F

from model import OCT
from utils import load_data
#We show how to use the pretrained checkpoint for inference in this notebook

In [2]:
#load the dataset and the model parameters here 
model = OCT(frame=512, embedding_dim=128, n_input_channels=60, n_conv_layers=3, kernel_size=3, stride=1, padding=1, pooling_kernel_size=3, pooling_stride=2, pooling_padding=1,
            dropout_rate=0.1, attention_dropout=0.1, stochastic_depth=0.1, num_layers=2, num_heads=1, num_classes=2, positional_embedding='learnable', mlp_ratio=1.0)

model.load_state_dict(torch.load('/root/ray_results/train_job_2021-11-03_08-50-24/train_job_15d07_00003_3_2021-11-03_08-50-24/checkpoint_000198/checkpoint')[0])
model.cuda()

_, test_ds=load_data()

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [3]:
f=open('new_score.txt', 'w')
for lfcc, target, name in test_ds:
    # way_loader.append(ways)
    lfcc, target=torch.from_numpy(lfcc).to(torch.float32).unsqueeze(dim=0).cuda().transpose(-1, -2), torch.tensor(target).unsqueeze(dim=0).cuda()
    with torch.no_grad():
        logits, feature=model(lfcc)
        score=torch.softmax(logits, dim=-1)[:, 0].item()            
    new_line=' '.join(name)+f' {score}\n'  #just append the score to the end of each line of the protocol file
    f.writelines(new_line)
    

f.close()